# Explore overlap between Neanderthal-introgressed SNPs and Nedelec eQTLs

This code explores if any Neanderthal-introgressed SNPs are present in the list of macrophage extreme response cis eQTLs identified by Nedelec *et al*. These QTLs comprise condition-specific eQTLs, response eQTLs (reQTLs) that are associated with gene expression levels after infection, and alternative splicing QTLs (asQTLs) that are associated with transcript isoform usage. Macrophages were subjected to 3 experimental conditions: *Listeria*-infected, *Salmonella*-infected and non-infected.

Neanderthal SNPs from:
1. Dannemann M, Prufer K & Kelso J. Functional implications of Neandertal introgression in modern humans. Genome Biol 2017 18:61.
2. Simonti CN *et al*. The phenotypic legacy of admixture between modern humans and Neandertals. Science 2016 351:737-41.

Macrophages eQTLs from:
1. Nedelec Y *et al*. Genetic Ancestry and Natural Selection Drive Population Differences in Immune Responses to Pathogens. Cell 2016 167:657-669 e21.

In [1]:
# Import libraries
import pandas as pd

In [2]:
# Load allpop_df containing Neanderthal SNPs
allpop_df = pd.read_csv("../neanderthal/allpop_df.csv", usecols=["Chromosome", "Position", "Source", "ID",
                                                  "ESN", "YRI", "MSL", "GWD", "LWK"])
allpop_df = allpop_df.dropna()

### Condition-specific eQTLs

In [3]:
# Load eQTL data
ned_eqtl = pd.read_excel("nedelec_eQTL.xlsx", "A) cis eQTL", skiprows=[0,1])
ned_eqtl.columns = ned_eqtl.iloc[0, :]
ned_eqtl.drop(ned_eqtl.index[0], inplace=True)
ned_eqtl.rename(columns={"external_gene_name": "Gene"}, inplace=True)

# Separate SNP IDs onto different rows based on treatment
ned_eqtl_1 = ned_eqtl.loc[:, ["Gene", "NI_top_snp_id"]]
ned_eqtl_1['Set'] = 'NI'
ned_eqtl_1.rename(columns={"NI_top_snp_id": "ID"}, inplace=True)
ned_eqtl_2 = ned_eqtl.loc[:, ["Gene", "L_top_snp_id"]]
ned_eqtl_2['Set'] = 'L'
ned_eqtl_2.rename(columns={"L_top_snp_id": "ID"}, inplace=True)
ned_eqtl_3 = ned_eqtl.loc[:, ["Gene", "S_top_snp_id"]]
ned_eqtl_3['Set'] = 'S'
ned_eqtl_3.rename(columns={"S_top_snp_id": "ID"}, inplace=True)
ned_eqtl = pd.concat([ned_eqtl_1, ned_eqtl_2, ned_eqtl_3])
ned_eqtl = ned_eqtl.sort_values(["Gene", "ID"], ascending=(True, True))

# Combine rows with same Gene and ID
ned_eqtl = ned_eqtl.groupby(["Gene", "ID"]).agg(','.join).reset_index()
ned_eqtl = ned_eqtl.groupby(["ID"]).agg(','.join).reset_index()

ned_eqtl.head()

,ID,Gene,Set
0,rs10001483,TBCK,S
1,rs10001849,PLAC8,NI
2,rs10001970,RPS3A,S
3,rs1000255,ST3GAL4,NI
4,rs1000354,FMNL1,S


In [4]:
# Intersecting SNPs in Neanderthal & Nedelec eQTL datasets
ned_eqtl_df = allpop_df.merge(ned_eqtl, how='inner', on=['ID'])
print("Number of intersecting SNPs: " + str(len(ned_eqtl_df)))
ned_eqtl_df.head()

Number of intersecting SNPs: 98


,Chromosome,Position,Source,ID,ESN,GWD,LWK,YRI,MSL,Gene,Set
0,1,54310748,both,rs7556609,0.000,0.000,0.000,0.000,0.000,YIPF1,NI
1,1,212429194,both,rs17018847,0.000,0.000,0.000,0.000,0.006,PPP2R5A,NI
2,10,64702053,simonti_only,rs2893913,0.020,0.018,0.045,0.023,0.029,EGR2,NI
3,11,9370093,both,rs4910456,0.000,0.000,0.000,0.000,0.000,IPO7,L
4,12,21568031,simonti_only,rs12322891,0.045,0.027,0.106,0.019,0.018,GOLT1B,L


In [5]:
# MAF for intersecting SNPs in African populations
ned_eqtl_df[['ESN', 'YRI', 'MSL', 'GWD', 'LWK']].describe()

,ESN,YRI,MSL,GWD,LWK
count,98.000000,98.000000,98.000000,98.000000,98.000000
mean,0.057490,0.056551,0.059204,0.060847,0.063112
std,0.220314,0.220687,0.219329,0.218350,0.219690
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.004000,0.005000
75%,0.005000,0.000000,0.012000,0.013000,0.018750
max,1.000000,1.000000,1.000000,0.996000,1.000000


In [6]:
# Number of SNPs with MAF > 0.01 in African populations
for pop in ['ESN', 'YRI', 'MSL', 'GWD', 'LWK']:
    df = ned_eqtl_df[ned_eqtl_df[pop] > 0.01]
    print('\033[1m' + pop + '\033[0m', 'Number: ' + str(len(df)), sep='\n')

ESN
Number: 18
YRI
Number: 17
MSL
Number: 27
GWD
Number: 26
LWK
Number: 28


In [7]:
# Expression context of intersecting SNPs
set = ned_eqtl_df['Set'].str.split(',', expand=True).stack().value_counts()
print('\033[1mExpression context of intersecting SNPs\033[0m', set, sep='\n')

genes = ned_eqtl_df['Gene'].str.split(',', expand=True).stack().value_counts()
print("\033[1mAssociated genes\033[0m", genes, sep='\n')

Expression context of intersecting SNPs
NI    44
L     35
S     31
dtype: int64
Associated genes
CYB5A        2
HPCA         2
SLC24A4      2
CCT2         2
NFKBIA       2
            ..
TUBB3        1
HIST1H2BJ    1
NRF1         1
SEC31A       1
REXO4        1
Length: 94, dtype: int64


### reQTLs

In [8]:
# Load reQTL data
ned_reqtl = pd.read_excel("nedelec_eQTL.xlsx", "B) cis reQTL", skiprows=[0,1])
ned_reqtl.columns = ned_reqtl.iloc[0, :]
ned_reqtl.drop(ned_reqtl.index[0], inplace=True)
ned_reqtl.rename(columns={"external_gene_name": "Gene"}, inplace=True)

# Separate SNP IDs onto different rows based on treatment
ned_reqtl_1 = ned_reqtl.loc[:, ["Gene", "FC_L_top_snp_id"]]
ned_reqtl_1['Set'] = 'L'
ned_reqtl_1.rename(columns={"FC_L_top_snp_id": "ID"}, inplace=True)
ned_reqtl_2 = ned_reqtl.loc[:, ["Gene", "FC_S_top_snp_id"]]
ned_reqtl_2['Set'] = 'S'
ned_reqtl_2.rename(columns={"FC_S_top_snp_id": "ID"}, inplace=True)
ned_reqtl = pd.concat([ned_reqtl_1, ned_reqtl_2])
ned_reqtl = ned_reqtl.sort_values(["Gene", "ID"], ascending=(True, True))

# Combine rows with same Gene and ID
ned_reqtl = ned_reqtl.groupby(["Gene", "ID"]).agg(','.join).reset_index()
ned_reqtl = ned_reqtl.groupby(["ID"]).agg(','.join).reset_index()

ned_reqtl.head()

,ID,Gene,Set
0,rs1000269,C20orf26,L
1,rs10003,IMPAD1,L
2,rs10005213,KLHL2,L
3,rs10005401,HNRNPDL,S
4,rs10008992,C4orf21,S


In [9]:
# Intersecting SNPs in Neanderthal & Nedelec reQTL datasets
ned_reqtl_df = allpop_df.merge(ned_reqtl, how='inner', on=['ID'])
print("Number of intersecting SNPs: " + str(len(ned_reqtl_df)))
ned_reqtl_df.head()

Number of intersecting SNPs: 53


,Chromosome,Position,Source,ID,ESN,GWD,LWK,YRI,MSL,Gene,Set
0,1,19892021,simonti_only,rs75061720,0.0,0.009,0.000,0.014,0.012,"MINOS1,NBL1","S,L"
1,11,36257328,simonti_only,rs61879262,0.0,0.000,0.000,0.000,0.000,COMMD9,S
2,12,54101479,both,rs73104021,0.0,0.000,0.005,0.000,0.000,CALCOCO1,L
3,13,73552634,simonti_only,rs75497275,0.0,0.004,0.000,0.000,0.000,KLF5,S
4,14,21004628,simonti_only,rs2319366,0.0,0.004,0.005,0.000,0.000,TMEM55B,L


In [10]:
# Expression context of intersecting SNPs
set = ned_reqtl_df['Set'].str.split(',', expand=True).stack().value_counts()
print('\033[1mExpression context of intersecting SNPs\033[0m', set, sep='\n')

genes = ned_reqtl_df['Gene'].str.split(',', expand=True).stack().value_counts()
print("\033[1mAssociated genes\033[0m", genes, sep='\n')

Expression context of intersecting SNPs
L    32
S    24
dtype: int64
Associated genes
CALCOCO1      2
E2F6          2
TMEM55B       1
OXNAD1        1
DUSP12        1
COMMD9        1
ALKBH7        1
PRDM1         1
MPP6          1
KLF5          1
CCDC146       1
KNSTRN        1
ITPR3         1
HIST2H2AA3    1
ACAP2         1
MINOS1        1
NR4A1         1
SNW1          1
PHRF1         1
HMGN4         1
CCDC42B       1
PNISR         1
TRAF3IP2      1
PPP1R37       1
PFDN6         1
AC187652.1    1
FAM120A       1
TRIM68        1
ELP4          1
IQCD          1
PSMD14        1
PTPDC1        1
ATP6V1C1      1
HECA          1
RAB11FIP2     1
PTPLAD2       1
MED11         1
ZNF318        1
TMEM41B       1
MIER3         1
MTFR1         1
ADCK1         1
RAB3IP        1
DDX21         1
RUVBL2        1
ZNF189        1
NBL1          1
FCGR2C        1
CCT2          1
FZD6          1
DENND5A       1
YTHDF1        1
dtype: int64


### asQTLs

In [11]:
# Load asQTL data
ned_asqtl = pd.read_excel("nedelec_eQTL.xlsx", "C) cis asQTL", skiprows=[0, 1])
ned_asqtl.columns = ned_asqtl.iloc[0, :]
ned_asqtl.drop(ned_asqtl.index[0], inplace=True)
ned_asqtl.rename(columns={"external_gene_name": "Gene"}, inplace=True)

# Separate SNP IDs onto different rows based on treatment
ned_asqtl_1 = ned_asqtl.loc[:, ["Gene", "NI_top_snp_id"]]
ned_asqtl_1['Set'] = 'NI'
ned_asqtl_1.rename(columns={"NI_top_snp_id": "ID"}, inplace=True)
ned_asqtl_2 = ned_asqtl.loc[:, ["Gene", "L_top_snp_id"]]
ned_asqtl_2['Set'] = 'L'
ned_asqtl_2.rename(columns={"L_top_snp_id": "ID"}, inplace=True)
ned_asqtl_3 = ned_asqtl.loc[:, ["Gene", "S_top_snp_id"]]
ned_asqtl_3['Set'] = 'S'
ned_asqtl_3.rename(columns={"S_top_snp_id": "ID"}, inplace=True)
ned_asqtl = pd.concat([ned_asqtl_1, ned_asqtl_2, ned_asqtl_3])

# Format dataframe
ned_asqtl = ned_asqtl.sort_values(["Gene", "ID"], ascending=(True, True))
ned_asqtl = ned_asqtl.drop_duplicates(subset=['Gene', 'ID', 'Set'], keep='first')
ned_asqtl = ned_asqtl.groupby(["Gene", "ID"]).agg(','.join).reset_index()
ned_asqtl = ned_asqtl.groupby(["ID"]).agg(','.join).reset_index()

ned_asqtl.head()

,ID,Gene,Set
0,rs10000006,"CYP2U1,SGMS2","NI,L,S,L"
1,rs10000012,CRIPAK,"NI,L,S"
2,rs1000002,MAP6D1,"NI,L,S"
3,rs10000037,TLR6,"NI,L,S"
4,rs10000130,"AC110615.1,CXCL10,CXCL11,CXCL9","NI,L,S,NI,L,S,NI,L,S,NI,L,S"


In [12]:
# Intersecting SNPs in Neanderthal & Nedelec asQTL datasets
ned_asqtl_df = allpop_df.merge(ned_asqtl, how='inner', on=['ID'])
print("Number of intersecting SNPs: " + str(len(ned_asqtl_df)))
ned_asqtl_df.head()

Number of intersecting SNPs: 333


,Chromosome,Position,Source,ID,ESN,GWD,LWK,YRI,MSL,Gene,Set
0,1,21658140,both,rs34693054,0.000,0.000,0.000,0.000,0.000,ECE1,S
1,1,22488940,simonti_only,rs2807362,0.005,0.009,0.040,0.009,0.000,CDC42,L
2,1,22502829,both,rs2744703,0.000,0.000,0.005,0.000,0.000,CDC42,NI
3,1,33229633,simonti_only,rs12022304,0.025,0.031,0.030,0.028,0.018,"RBBP4,YARS","NI,NI"
4,1,33443064,both,rs735402,0.000,0.027,0.040,0.000,0.029,AK2,L


In [13]:
# Expression context of intersecting SNPs
set = ned_asqtl_df['Set'].str.split(',', expand=True).stack().value_counts()
print('\033[1mExpression context of intersecting SNPs\033[0m', set, sep='\n')

genes = ned_asqtl_df['Gene'].str.split(',', expand=True).stack().value_counts()
print("\033[1mAssociated genes\033[0m", genes, sep='\n')

Expression context of intersecting SNPs
NI    147
S     135
L     121
dtype: int64
Associated genes
RAD1        3
FCGR2A      3
DDX60L      3
OAT         3
SAMSN1      3
           ..
TGFBRAP1    1
BRD9        1
LHPP        1
AHNAK       1
PSEN1       1
Length: 309, dtype: int64
